<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-необходимых-библиотек-и-данных" data-toc-modified-id="Импорт-необходимых-библиотек-и-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт необходимых библиотек и данных</a></span></li><li><span><a href="#Загрузка-датасета" data-toc-modified-id="Загрузка-датасета-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка датасета</a></span></li><li><span><a href="#Лемматиация-данных" data-toc-modified-id="Лемматиация-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Лемматиация данных</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Модель-LGBM" data-toc-modified-id="Модель-LGBM-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Модель LGBM</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span><ul class="toc-item"><li><span><a href="#Сводная-таблица" data-toc-modified-id="Сводная-таблица-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Сводная таблица</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Цель и задачи

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель:**
- Обучить модель классифицировать комментарии на позитивные и негативные. 

**Задачи**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.


# Подготовка

## Импорт необходимых библиотек и данных

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import pickle
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка датасета

In [2]:
try:
    df = pd.read_csv('C:/Users/Oblre/Downloads/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

## Лемматиация данных

Для удобства обработки данных создана функция для лемматизации столбца, состоящего из текста. Там же избавились от лишних символов с помощью модуля `re (regular expression)` и от лишних пробелов с помощью `split` и `join`.

In [3]:
m = WordNetLemmatizer()
def lemmatize(X):
    x = []
    for row in X:
        lemm_list = m.lemmatize(row)
        lemm_text = "".join(lemm_list)
        clean = re.sub(r'[^a-zA-zа-яА-яёЁ ]', ' ', lemm_text)
        joined = ' '.join(clean.split())
        x.append(joined)
    return x

In [4]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Проверили датасет на корректность вышеперечисленных операций.

In [5]:
df['text'] = lemmatize(df['text'])
df

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He matches this background colour I m se...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0
...,...,...
159566,And for the second time of asking when your vi...,0
159567,You should be ashamed of yourself That is a ho...,0
159568,Spitzer Umm theres no actual article for prost...,0
159569,And it looks like it was actually you who put ...,0


Разделили данные на тренировочную(75%) и тестовую(25%) выборки.  

In [6]:
X = df.drop('toxic', axis=1) 
y = df['toxic'] 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

С помощью функции `TfidfVectorizer` создали признаки (векторы величин `TF-IDF`) из твитов для дальнейшего обучения. Далее, с помощью функции fit обучили только тренировочную выборку (чтобы тестирвоание было честным), а потом трансформировали тренирвоочную и тестовую выборки.   

In [7]:
stopwords = set(stopwords.words('english', 'russian'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

x_train = x_train['text'].values.astype('U')
x_test = x_test['text'].values.astype('U')
tf_idf = count_tf_idf.fit(x_train)
x_train = count_tf_idf.transform(x_train)
x_test = count_tf_idf.transform(x_test)

## Обучение

### Логистическая регрессия 

In [8]:
model_lr = LogisticRegression().fit(x_train, y_train)
pred_test_lr = model_lr.predict(x_test)
f1_score_lr_1 = f1_score(y_test, pred_test_lr)
f1_score_lr_2 = f1_score(y_test, pred_test_lr, average='weighted')
print('Показатель F1_score для логистической регрессии: {:.2f}'.format(f1_score_lr_1))
print('Показатель F1_score для логистической регрессии (при average=weighted): {:.2f}'.format(f1_score_lr_2))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Показатель F1_score для логистической регрессии: 0.73
Показатель F1_score для логистической регрессии (при average=weighted): 0.95


<div class="alert alert-block alert-info">

Лучший результат метрики с параметром (average=weighted) `F1_score = 0.95`. Результат улучшается засчет параметра (average=weighted), с параметрами по умолчанию `F1_score = 0.73`. 

Данный результат удовлетворяет исходным условиям, где F1_score = 0.75, но при парамтере (average=weighted).

### Дерево решений

In [9]:
best_model_dt = None
best_f1_score_DT = 0

for depth in range(100, 151, 10):
    model_DT = DecisionTreeClassifier(random_state=0, max_depth=depth)
    model_DT.fit(x_train, y_train)
    pred_test_DT = model_DT.predict(x_test)
    f1_score_DT = f1_score(y_test, pred_test_DT)
    if f1_score_DT > best_f1_score_DT:
        best_model_dt = model_DT
        best_f1_score_DT = f1_score_DT
            
print('Лучшая модель со след. параметрами:', '\n', best_model_dt)
print('Показатель F1_score для дерева решений: {:.2f}'.format(best_f1_score_DT))

Лучшая модель со след. параметрами: 
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=140,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')
Показатель F1_score для дерева решений: 0.73


In [10]:
best_model_dt_1 = None
best_f1_score_DT_1 = 0

for depth in range(100, 151, 10):
    model_DT = DecisionTreeClassifier(random_state=0, max_depth=depth)
    model_DT.fit(x_train, y_train)
    pred_test_DT = model_DT.predict(x_test)
    f1_score_DT = f1_score(y_test, pred_test_DT, average='weighted')
    if f1_score_DT > best_f1_score_DT_1:
        best_model_dt_1 = model_DT
        best_f1_score_DT_1 = f1_score_DT
            
print('Лучшая модель со след. параметрами:', '\n', best_model_dt_1)
print('Показатель F1_score для дерева решений: {:.2f}'.format(best_f1_score_DT_1))

Лучшая модель со след. параметрами: 
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')
Показатель F1_score для дерева решений: 0.95


<div class="alert alert-block alert-info">

Лучший гиперпараметр для данной модели: `max_depth = 100` с результатом метрики `F1_score = 0.95` при введение параметра average.

Данный результат не удовлетворяет исходным условиям, где F1_score = 0.75.

### Случайный лес

Модель с подбором гиперпараметров:

In [11]:
best_model_rf = None
best_f1_score_RF = 0

for est in range(3, 10, 1):
    for depth in range(1, 100, 10):
        model_RF = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model_RF.fit(x_train, y_train)
        pred_test_RF = model_RF.predict(x_test)
        f1_score_RF = f1_score(y_test, pred_test_RF)
        if f1_score_RF > best_f1_score_RF:
            best_model_rf = model_RF
            best_f1_score_RF = f1_score_RF

print('Лучшая модель со след. параметрами:', '\n', best_model_rf)
print('Показатель F1_score для случайного леса (с подбором гиперпараметров): {:.2f}'.format(best_f1_score_RF))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-sco

Лучшая модель со след. параметрами: 
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=91, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
Показатель F1_score для случайного леса (с подбором гиперпараметров): 0.39


Модель без подбора гиперпараметров:

In [12]:
model_RF_1 = RandomForestClassifier(random_state=0)
model_RF_1.fit(x_train, y_train)
pred_test_RF_1 = model_RF_1.predict(x_test)
f1_score_RF_1 = f1_score(y_test, pred_test_RF_1)
print('Показатель F1_score для случайного леса (без подбора гиперпараметров): {:.2f}'.format(f1_score_RF_1))
f1_score_RF_2 = f1_score(y_test, pred_test_RF_1, average='weighted')
print('Показатель F1_score для случайного леса (без подбора гиперпараметров, но с average=weighted): {:.2f}'.format(f1_score_RF_2))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Показатель F1_score для случайного леса (без подбора гиперпараметров): 0.67
Показатель F1_score для случайного леса (без подбора гиперпараметров, но с average=weighted): 0.94


<div class="alert alert-block alert-info">

Лучший результат метрики получился без подбора гиперпараметров, но с использованием параметра average `F1_score = 0.95`. Данный результат удовлетворяет исходным условиям, где F1_score = 0.75.
    
При гиперпараметрах: max_depth=91, n_estimators=4 `F1_score = 0.39`

### Модель LGBM

In [13]:
best_model_lgbm = None
best_f1_score_LGBM = 0
 
for est in range(1, 71, 10):
    for depth in range(1, 5, 1):
        model_lgbm = LGBMClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model_lgbm.fit(x_train,y_train)
        pred_test_LGBM = model_lgbm.predict(x_test)
        f1_score_LGBM = f1_score(y_test, pred_test_LGBM)
        if f1_score_LGBM > best_f1_score_LGBM:
            best_model_lgbm = model_lgbm
            best_f1_score_LGBM = f1_score_LGBM

print('Лучшая модель со след. параметрами:', '\n', best_model_lgbm)
print('Показатель F1_score для LGBM (average=macro): {:.2f}'.format(best_f1_score_LGBM))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Лучшая модель со след. параметрами: 
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=4,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=61, n_jobs=-1, num_leaves=31, objective=None,
               random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
Показатель F1_score для LGBM (average=macro): 0.55


In [14]:
best_model_lgbm_1 = None
best_f1_score_LGBM_1 = 0
 
for est in range(1, 71, 10):
    for depth in range(1, 5, 1):
        model_lgbm = LGBMClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model_lgbm.fit(x_train,y_train)
        pred_test_LGBM = model_lgbm.predict(x_test)
        f1_score_LGBM = f1_score(y_test, pred_test_LGBM, average='macro')
        if f1_score_LGBM > best_f1_score_LGBM_1:
            best_model_lgbm_1 = model_lgbm
            best_f1_score_LGBM_1 = f1_score_LGBM

print('Лучшая модель со след. параметрами:', '\n', best_model_lgbm_1)
print('Показатель F1_score для LGBM (average=macro): {:.2f}'.format(best_f1_score_LGBM_1))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Лучшая модель со след. параметрами: 
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=4,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=61, n_jobs=-1, num_leaves=31, objective=None,
               random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
Показатель F1_score для LGBM (average=macro): 0.76


<div class="alert alert-block alert-info">

Лучший результат модели с гиперпараметрами: max_depth=4, n_estimators=61 и метрики с параметром (average=macro) `F1_score = 0.76`. Результат улучшается засчет параметра (average=macro), с параметрами по умолчанию `F1_score = 0.55`. 

Данный результат удовлетворяет исходным условиям, где F1_score = 0.75, но при парамтере (average=macro).

## Выводы

### Сводная таблица

In [15]:
result_name = ['model_name', 'F1_score', 'F1_score(average=...)']
result_lr = ['LogisticRegression', f1_score_lr_1, f1_score_lr_2]
result_dt = ['DesicionTreeClassifier', best_f1_score_DT, best_f1_score_DT_1]
result_rf = ['RandomForestClassifier', f1_score_RF_1, f1_score_RF_2]
result_lgbm = ['LightGBMClassifier', best_f1_score_LGBM, best_f1_score_LGBM_1]

df_results = pd.DataFrame([result_lr, result_dt, result_rf, result_lgbm], columns=result_name)

df_results = df_results.round(2)
df_results.style.set_caption("Results of the different machine learning models")
df_results = df_results.sort_values('F1_score(average=...)', ascending=True).reset_index()
df_results.drop('index', axis=1, inplace=True)

df_results

,model_name,F1_score,F1_score(average=...)
0,LightGBMClassifier,0.55,0.76
1,RandomForestClassifier,0.67,0.94
2,LogisticRegression,0.73,0.95
3,DesicionTreeClassifier,0.73,0.95


<div class="alert alert-block alert-info">

**В проекте были выполнены следующие действия:**
    
  1. Проведена предобработка данных, где были лемматизированы текстовые данные, очищены от лишних символов и соединены без лишних пропусков.  
  2. Данные были приведены к общему типу для дальнейшего создания признаков на основе уникальных слов. 
  3. Были обучены четыре классификационные модели машинного обучения, а также подобраны оптимальные параметры для их обучения.
  4. Получены прогнозируемые значения для характера твита.
  5. Исходя из прогнозов была рассчитана метрика: F1, которая выражается в долях.
 
**Исходя из полученных данных можно сделать следующий вывод:**   
Наиболее эффективными моделями по полученным результатам оказались модели LogisticRegression, DesicionTreeClassifier и RandomForestClassifier с недефолтными параметрами (с `average=weighted`). **Значение F1 составило 0.95**. Это наибольший показатель среди всех моделей, но с подобранным параметром `average` у метрики F1_score. Данный подобранный параметр может искажать суть метрики, так как зависит от среднезвешенного среднего.
Наиболее эффективными моделями по полученным результатам оказались модели LogisticRegression, DesicionTreeClassifier с дефолтными параметрами. **Значение F1 составило 0.73**.
Значение 0.95 не выглядит правдоподобно, так как при введение `average=weighted` любая метрика дает одинаковый высокий результат.
На мой взгляд лучший показатель у модели - LightGBMClassifier с метрикой **F1 = 0.76** при `average=macro`. `average=macro` не зависит от баланса классов, поэтому результат похож на правду.